In [ ]:
import os
import csv
import sqlite3

import igraph

In [ ]:
path = os.path.abspath(os.path.expanduser(input('Read to csv file: ').strip()))

In [ ]:
dbpath = 'index.db'
conn = sqlite3.connect(dbpath)
cur = conn.cursor()

def get_blkid(blkhash):
    cur.execute('''SELECT id FROM BlkID WHERE blkhash = ?''', (blkhash,))
    return cur.fetchone()[0]
    
def get_txid(txhash):
    cur.execute('''SELECT id FROM TxID WHERE txhash = ?''', (txhash,))
    return cur.fetchone()[0]
    
def get_addrid(addr):
    cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (addr,))
    return cur.fetchone()[0]

def get_txins(addr):
    cur.execute('''SELECT tx FROM TxIn WHERE addr = ?''', (addr,))
    return cur.fetchall()

def get_txouts(addr):
    cur.execute('''SELECT tx FROM TxOut WHERE addr = ?''', (addr,))
    return cur.fetchall()

def get_tx(saddr, daddr):
    cur.execute('''SELECT tx FROM TxIn WHERE addr = ?
                   INTERSECT
                   SELECT tx FROM TxOut WHERE addr = ?;''', (saddr, daddr))
    r = cur.fetchone()
    if r is not None:
        return True
    return False

In [ ]:
nodes = list()
with open(path, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        addrid = get_addrid(row['Address'])
        nodes.append(addrid)

In [ ]:
edges = list()
for s in nodes:
    for d in nodes:
        if get_tx(s, d):
            edges.append((s, d))
            print(f'Edge {s} - {d}', end='\r')

In [ ]:
import time
stime = time.time()
partition = g.community_leiden(objective_function='modularity')
etime = time.time()
size = len(partition)
print(f'{size} {etime-stime}')